In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from salesPrediction.constants import *
from salesPrediction.utils.common import read_yaml, create_directories, get_complete_schema_structure
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib


In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_path= CONFIG_FILE_PATH,
                 params_path= PARAMS_FILE_PATH,
                 schema_path= SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        target_column = list(self.schema.TARGET_COLUMN.keys())[0]


        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=target_column
        )

        return model_trainer_config

        

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def Train(self):
        train_schema, test_schema, tranformed_cols_to_drop = get_complete_schema_structure(SCHEMA_FILE_PATH)

        train_data = pd.read_csv(self.config.train_data_path, dtype=train_schema)
        test_data = pd.read_csv(self.config.test_data_path, dtype=test_schema)

        train_data.drop(columns=[col for col in tranformed_cols_to_drop if col in train_data.columns], axis=1, inplace=True)
        test_data.drop(columns=[col for col in tranformed_cols_to_drop if col in test_data.columns], axis=1, inplace=True)

        train_x  = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]] 

        print(train_x.dtypes)
        print(train_y.dtypes)

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.Train()
    
except Exception as e:
    raise(e)

[2025-03-21 19:17:20,673: common:yaml file: config\config.yaml loaded successfully]
[2025-03-21 19:17:20,676: common:yaml file: params.yaml loaded successfully]
[2025-03-21 19:17:20,679: common:yaml file: schema.yaml loaded successfully]
[2025-03-21 19:17:20,681: common:created directory at artifacts]
[2025-03-21 19:17:20,681: common:created directory at artifacts/data_transformation]
[2025-03-21 19:17:20,685: common:yaml file: schema.yaml loaded successfully]
{'Item_Identifier': 'string', 'Item_Weight': 'float64', 'Item_Fat_Content': 'object', 'Item_Visibility': 'float64', 'Item_Type': 'object', 'Item_MRP': 'float64', 'Outlet_Identifier': 'string', 'Outlet_Establishment_Year': 'int64', 'Outlet_Size': 'object', 'Outlet_Location_Type': 'object', 'Outlet_Type': 'object', 'Item_Outlet_Sales': 'float64'}
artifacts/data_transformation/Train.csv
Item_Weight                             float64
Item_Fat_Content                         object
Item_Visibility                         float64
Item